# SWaT Dataset Preprocessing

In [52]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
import pathlib

In [58]:
root_dir = '../data'
data_list = [os.path.join(root_dir, f) for f in os.listdir(root_dir)]
data_list

['../data/SWaT_Dataset_Attack_v0.xlsx',
 '../data/kdd99_test.npy',
 '../data/SWaT_Dataset_Normal_v1.xlsx',
 '../data/SWaT_Dataset_Normal_v0.xlsx',
 '../data/kdd99_train.npy']

In [25]:
att_df = pd.read_excel(data_list[0], sheet_name='Combined Data', header=1)
att_df.head()

/Users/innohub/bin/miniconda3/envs/gan/lib/python3.10/site-packages/openpyxl/worksheet/_read_only.py:81: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():


,Timestamp,FIT101,LIT101,MV101,P101,P102,AIT201,AIT202,AIT203,FIT201,...,P501,P502,PIT501,PIT502,PIT503,FIT601,P601,P602,P603,Normal/Attack
0,28/12/2015 10:00:00 AM,2.427057,522.8467,2,2,1,262.0161,8.396437,328.6337,2.445391,...,2,1,250.8652,1.649953,189.5988,0.000128,1,1,1,Normal
1,28/12/2015 10:00:01 AM,2.446274,522.8860,2,2,1,262.0161,8.396437,328.6337,2.445391,...,2,1,250.8652,1.649953,189.6789,0.000128,1,1,1,Normal
2,28/12/2015 10:00:02 AM,2.489191,522.8467,2,2,1,262.0161,8.394514,328.6337,2.442316,...,2,1,250.8812,1.649953,189.6789,0.000128,1,1,1,Normal
3,28/12/2015 10:00:03 AM,2.534350,522.9645,2,2,1,262.0161,8.394514,328.6337,2.442316,...,2,1,250.8812,1.649953,189.6148,0.000128,1,1,1,Normal
4,28/12/2015 10:00:04 AM,2.569260,523.4748,2,2,1,262.0161,8.394514,328.6337,2.443085,...,2,1,250.8812,1.649953,189.5027,0.000128,1,1,1,Normal


In [30]:
print(att_df.isnull().sum())

cols = att_df.columns
print(f'Columns: {cols}')
att_df.columns = cols.str.strip()
print(f'After process: {att_df.columns}, shape: {att_df.shape}')

Timestamp        0
FIT101           0
LIT101           0
MV101            0
P101             0
P102             0
AIT201           0
AIT202           0
AIT203           0
FIT201           0
MV201            0
P201             0
P202             0
P203             0
P204             0
P205             0
P206             0
DPIT301          0
FIT301           0
LIT301           0
MV301            0
MV302            0
MV303            0
MV304            0
P301             0
P302             0
AIT401           0
AIT402           0
FIT401           0
LIT401           0
P401             0
P402             0
P403             0
P404             0
UV401            0
AIT501           0
AIT502           0
AIT503           0
AIT504           0
FIT501           0
FIT502           0
FIT503           0
FIT504           0
P501             0
P502             0
PIT501           0
PIT502           0
PIT503           0
FIT601           0
P601             0
P602             0
P603             0
Normal/Attac

In [32]:
class_counts = att_df['Normal/Attack'].value_counts()
print(class_counts)

att_df['Normal/Attack'] = att_df['Normal/Attack'].replace('A ttack', 'Attack')
class_counts = att_df['Normal/Attack'].value_counts()
print(f'Process label:\n{class_counts}')
assert class_counts['Normal'] + class_counts['Attack'] == att_df.shape[0]


Normal/Attack
Normal    395298
Attack     54621
Name: count, dtype: int64
Process label:
Normal/Attack
Normal    395298
Attack     54621
Name: count, dtype: int64


In [44]:
from datetime import datetime
import pytz

def convert_to_timestamp(date_string):
    dt = datetime.strptime(date_string.strip(), '%d/%m/%Y %I:%M:%S %p')
    return int(dt.replace(tzinfo=pytz.UTC).timestamp())

timestamp = convert_to_timestamp('28/12/2015 10:00:00 AM')
print(timestamp)

1451296800


In [45]:
att_df.loc[:, 'label'] = att_df['Normal/Attack'].map({'Normal': 0, 'Attack': 1})
att_df.loc[:, 'TS-UTC'] = att_df['Timestamp'].apply(lambda x: convert_to_timestamp(x))
print(f'Class label:\n{att_df["label"].value_counts()}')
att_df.head()

Class label:
label
0    395298
1     54621
Name: count, dtype: int64


,Timestamp,FIT101,LIT101,MV101,P101,P102,AIT201,AIT202,AIT203,FIT201,...,PIT501,PIT502,PIT503,FIT601,P601,P602,P603,Normal/Attack,label,TS-UTC
0,28/12/2015 10:00:00 AM,2.427057,522.8467,2,2,1,262.0161,8.396437,328.6337,2.445391,...,250.8652,1.649953,189.5988,0.000128,1,1,1,Normal,0,1451296800
1,28/12/2015 10:00:01 AM,2.446274,522.8860,2,2,1,262.0161,8.396437,328.6337,2.445391,...,250.8652,1.649953,189.6789,0.000128,1,1,1,Normal,0,1451296801
2,28/12/2015 10:00:02 AM,2.489191,522.8467,2,2,1,262.0161,8.394514,328.6337,2.442316,...,250.8812,1.649953,189.6789,0.000128,1,1,1,Normal,0,1451296802
3,28/12/2015 10:00:03 AM,2.534350,522.9645,2,2,1,262.0161,8.394514,328.6337,2.442316,...,250.8812,1.649953,189.6148,0.000128,1,1,1,Normal,0,1451296803
4,28/12/2015 10:00:04 AM,2.569260,523.4748,2,2,1,262.0161,8.394514,328.6337,2.443085,...,250.8812,1.649953,189.5027,0.000128,1,1,1,Normal,0,1451296804


## Normal Data

In [46]:
nor_df0 = pd.read_excel(data_list[2], sheet_name='Normal.csv', header=1)
print(nor_df0.head())

/Users/innohub/bin/miniconda3/envs/gan/lib/python3.10/site-packages/openpyxl/worksheet/_read_only.py:81: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():


                Timestamp    FIT101    LIT101  MV101  P101  P102    AIT201  \
0   22/12/2015 4:00:00 PM  2.470294  261.5804      2     2     1  244.3284   
1   22/12/2015 4:00:01 PM  2.457163  261.1879      2     2     1  244.3284   
2   22/12/2015 4:00:02 PM  2.439548  260.9131      2     2     1  244.3284   
3   22/12/2015 4:00:03 PM  2.428338  260.2850      2     2     1  244.3284   
4   22/12/2015 4:00:04 PM  2.424815  259.8925      2     2     1  244.4245   

    AIT202   AIT203    FIT201  ...  P501  P502    PIT501  PIT502    PIT503  \
0  8.19008  306.101  2.471278  ...     1     1  10.02948     0.0  4.277749   
1  8.19008  306.101  2.468587  ...     1     1  10.02948     0.0  4.277749   
2  8.19008  306.101  2.467305  ...     1     1  10.02948     0.0  4.277749   
3  8.19008  306.101  2.466536  ...     1     1  10.02948     0.0  4.277749   
4  8.19008  306.101  2.466536  ...     1     1  10.02948     0.0  4.277749   

     FIT601  P601  P602  P603  Normal/Attack  
0  0.000256    

In [48]:
print(f'Shape: {nor_df0.shape}, cols: {nor_df0.columns}')
print(f'Null values:\n{nor_df0.isnull().sum()}')

Shape: (496800, 53), cols: Index([' Timestamp', 'FIT101', 'LIT101', 'MV101', 'P101', 'P102', 'AIT201',
       'AIT202', 'AIT203', 'FIT201', 'MV201', 'P201', 'P202', 'P203', 'P204',
       'P205', 'P206', 'DPIT301', 'FIT301', 'LIT301', 'MV301', 'MV302',
       'MV303', 'MV304', 'P301', 'P302', 'AIT401', 'AIT402', 'FIT401',
       'LIT401', 'P401', 'P402', 'P403', 'P404', 'UV401', 'AIT501', 'AIT502',
       'AIT503', 'AIT504', 'FIT501', 'FIT502', 'FIT503', 'FIT504', 'P501',
       'P502', 'PIT501', 'PIT502', 'PIT503', 'FIT601', 'P601', 'P602', 'P603',
       'Normal/Attack'],
      dtype='object')
Null values:
 Timestamp       0
FIT101           0
LIT101           0
MV101            0
P101             0
P102             0
AIT201           0
AIT202           0
AIT203           0
FIT201           0
MV201            0
P201             0
P202             0
P203             0
P204             0
P205             0
P206             0
DPIT301          0
FIT301           0
LIT301           0
MV30

In [51]:
class_counts = nor_df0['Normal/Attack'].value_counts()
print(f'Class counts:\n{class_counts}')

def preprocess_df(df):
    assert df.isnull().sum().sum() == 0
    df.columns = df.columns.str.strip()
    df['Normal/Attack'] = df['Normal/Attack'].replace('\s+', '', regex=True)
    df.loc[:, 'label'] = df['Normal/Attack'].map({'Normal': 0, 'Attack': 1})
    df.loc[:, 'TS-UTC'] = df['Timestamp'].apply(lambda x: convert_to_timestamp(x))
    return df

preprocess_df(nor_df0)
nor_df0.head()

Class counts:
Normal/Attack
Normal    496800
Name: count, dtype: int64


,Timestamp,FIT101,LIT101,MV101,P101,P102,AIT201,AIT202,AIT203,FIT201,...,PIT501,PIT502,PIT503,FIT601,P601,P602,P603,Normal/Attack,label,TS-UTC
0,22/12/2015 4:00:00 PM,2.470294,261.5804,2,2,1,244.3284,8.19008,306.101,2.471278,...,10.02948,0.0,4.277749,0.000256,1,1,1,Normal,0,1450800000
1,22/12/2015 4:00:01 PM,2.457163,261.1879,2,2,1,244.3284,8.19008,306.101,2.468587,...,10.02948,0.0,4.277749,0.000256,1,1,1,Normal,0,1450800001
2,22/12/2015 4:00:02 PM,2.439548,260.9131,2,2,1,244.3284,8.19008,306.101,2.467305,...,10.02948,0.0,4.277749,0.000256,1,1,1,Normal,0,1450800002
3,22/12/2015 4:00:03 PM,2.428338,260.2850,2,2,1,244.3284,8.19008,306.101,2.466536,...,10.02948,0.0,4.277749,0.000256,1,1,1,Normal,0,1450800003
4,22/12/2015 4:00:04 PM,2.424815,259.8925,2,2,1,244.4245,8.19008,306.101,2.466536,...,10.02948,0.0,4.277749,0.000256,1,1,1,Normal,0,1450800004


# KDD99 Data

In [57]:
kdd = np.load(data_list[-1])
print(kdd.shape)
print(kdd[:1])

(562387, 35)
[[0.0000e+00 2.1500e+02 4.5076e+04 0.0000e+00 0.0000e+00 0.0000e+00
  0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00
  0.0000e+00 0.0000e+00 0.0000e+00 1.0000e+00 1.0000e+00 0.0000e+00
  0.0000e+00 0.0000e+00 0.0000e+00 1.0000e+00 0.0000e+00 0.0000e+00
  0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00
  0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00]]
